# 🔍 Walk-Forward 参数优化

本笔记本展示如何使用 Walk-Forward 验证优化策略参数。

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ml_framework import FeatureEngineer, WalkForwardOptimizer

## 什么是 Walk-Forward 优化？

Walk-Forward 优化是一种时间序列交叉验证方法：

1. **避免未来信息泄露**: 只使用历史数据训练
2. **多折验证**: 分成多个时间段测试
3. **稳健性检验**: 确保参数在不同时期都有效

### 优化流程

```
折1: |--训练--|--测试--|
折2:     |--训练--|--测试--|
折3:         |--训练--|--测试--|
折4:             |--训练--|--测试--|
折5:                 |--训练--|--测试--|
```

## 1. 准备数据

In [ ]:
# 加载数据
data_path = Path.cwd().parent / 'data' / 'TOPIX_18months_clean.csv'
if data_path.exists():
    df = pd.read_csv(data_path, nrows=50000)
    df['datetime'] = pd.to_datetime(df['datetime'])
    print(f'✅ 加载 {len(df):,} 行真实数据')
else:
    dates = pd.date_range('2024-01-01', periods=10000, freq='1min')
    df = pd.DataFrame({
        'datetime': dates,
        'open': 2400 + np.cumsum(np.random.randn(10000) * 2),
        'high': 2400 + np.cumsum(np.random.randn(10000) * 2) + 5,
        'low': 2400 + np.cumsum(np.random.randn(10000) * 2) - 5,
        'close': 2400 + np.cumsum(np.random.randn(10000) * 2),
        'volume': np.random.randint(1000, 50000, 10000)
    })
    print('⚠️ 使用示例数据')

In [ ]:
# 创建特征
engineer = FeatureEngineer()
df = engineer.create_all_features(df)
df = df.dropna()
print(f'特征准备完成: {len(df):,} 行')

## 2. 定义参数空间

In [ ]:
# 参数网格（小规模演示）
param_grid = {
    'profit_threshold': [0.003, 0.005, 0.008],  # 标签生成阈值
    'lookahead_bars': [15, 20, 30],              # 向前看的K线数
    'ml_confidence': [0.15, 0.20, 0.25],         # ML置信度阈值
    'tp_multiplier': [2.0, 3.0, 4.0],            # 止盈倍数
    'sl_multiplier': [0.8, 1.0, 1.2],            # 止损倍数
    'trail_multiplier': [1.0, 1.2, 1.5]          # 追踪止损倍数
}

total_combinations = 1
for values in param_grid.values():
    total_combinations *= len(values)

print(f'参数组合总数: {total_combinations}')
print(f'\n参数范围:')
for key, values in param_grid.items():
    print(f'  {key}: {values}')

## 3. 运行优化

⚠️ **注意**: 完整优化需要较长时间（3-4小时）。这里使用小规模参数演示。

In [ ]:
# 创建优化器
optimizer = WalkForwardOptimizer(
    n_splits=3,              # 3折交叉验证（演示用）
    min_trades=10,           # 最少交易数
    initial_capital=5_000_000,
    tick_size=0.5,
    multiplier=10000
)

print('⏳ 开始优化...')
print('   这可能需要几分钟...')

In [ ]:
# 运行优化
best = optimizer.optimize(df, param_grid, verbose=True)

## 4. 分析结果

In [ ]:
# Top 5 参数集
top_5 = optimizer.get_top_n(5)

print('Top 5 参数组合:')
print('='*70)
for i, result in enumerate(top_5, 1):
    print(f'\n排名 {i}:')
    print(f'  平均 ROI: {result["avg_roi"]:.2f}%')
    print(f'  标准差:   {result["std_roi"]:.2f}%')
    print(f'  最小 ROI: {result["min_roi"]:.2f}%')
    print(f'  最大 ROI: {result["max_roi"]:.2f}%')
    print(f'  有效折数: {result["valid_folds"]}')
    print(f'  参数:')
    for key, value in result['params'].items():
        print(f'    {key}: {value}')

## 5. 可视化优化结果

In [ ]:
# 绘制优化结果
optimizer.plot_results(top_n=20)

## 6. 保存结果

In [ ]:
# 保存优化结果
output_path = Path.cwd().parent / 'results' / 'optimization_results.json'
optimizer.save_results(str(output_path))

## 💡 关键要点

1. **时间序列验证**: Walk-Forward 避免未来信息泄露
2. **多折验证**: 确保参数在不同时期稳健
3. **参数敏感性**: 观察哪些参数对结果影响最大
4. **过拟合风险**: 验证集之间差异大说明可能过拟合

## 下一步

- **05_performance_analysis.ipynb** - 详细性能分析
- 使用最佳参数进行实盘测试